# [Sign language dactyl recognition based on machine learning algorithms](https://www.researchgate.net/publication/354349413_Sign_language_dactyl_recognition_based_on_machine_learning_algorithms)

In [ ]:
import numpy as np 
import pandas as pd 
import glob
import os
import cv2
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import tensorflow as tf
from PIL import Image
import keras
from sklearn.preprocessing import LabelEncoder


In [ ]:
TEST_DATA_PATH = '/Users/kamilla/Desktop/masters_diploma/SOTA_MODELS/data/croped_test_29_04_cv'
TRAIN_DATA_PATH = '/Users/kamilla/Desktop/masters_diploma/SOTA_MODELS/data/croped_train_29_04_cv'
#VAL_DATA_PATH = '/Users/kamilla/Desktop/masters_diploma/SOTA_MODELS/SOTA_3/data_SOTA_3/temp_val'

# Создадим датафрейм, где будем сохранять путь до картинки и ее лейбл

dactyl = ["А", "Б", "В", "Г", "Е",
          "Ж", "И", "К", "Л", "М",
          "Н", "О", "П", "Р", "С",
          "Т", "У", "Ф", "Х", "Ч",
          "Ш", "Ы", "Э", "Ю", "Я"]

def get_metadata(data_path):
    list_path = []
    list_labels = []
    for label in dactyl:
        label_path = os.path.join(data_path, label, "*")
        image_files = glob.glob(label_path)

        sign_label = [label] * len(image_files)

        list_path.extend(image_files)
        list_labels.extend(sign_label)

    metadata = pd.DataFrame({
        "image_path": list_path,
        "label": list_labels
    })
    return metadata

train_df = get_metadata(TRAIN_DATA_PATH)
print(f"DataFrame for train is ready. Processed {len(train_df)}")
test_df = get_metadata(TEST_DATA_PATH)
print(f"DataFrame for test is ready. Processed {len(test_df)}")
#val_df = get_metadata(VAL_DATA_PATH)
#print(f"DataFrame for val is ready. Processed {len(val_df)}")

In [ ]:
y_train, x_train = train_df['label'], train_df['image_path']
y_test, x_test = test_df['label'], test_df['image_path']
#y_val, x_val = val_df['label'], val_df['image_path']


In [ ]:
le = LabelEncoder()
le.fit(y_train)
y_train_encoded = le.transform(y_train)
y_train_cat = keras.utils.to_categorical(y_train_encoded)

y_test_encoded = le.transform(y_test)
y_test_cat = keras.utils.to_categorical(y_test_encoded)

#y_val_encoded = le.transform(y_val_prep)
#y_val_cat = keras.utils.to_categorical(y_val_encoded)

In [ ]:
x_train_images[0]

In [ ]:
def load_and_preprocess_image(image_path, y_data, target_size=(32, 32), augmentation=False):
    skin_segmentation_images = []
    labels = []
    if image_path.endswith(".jpg") or image_path.endswith(".jpeg"):
        image = cv2.imread(image_path)        
        hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

        # Define skin color range in HSV
        lower_skin = np.array([0, 0, 0]) 
        upper_skin = np.array([38, 255, 255]) 

        skin_mask = cv2.inRange(hsv_image, lower_skin, upper_skin)
        skin_extracted_image = cv2.bitwise_and(image, image, mask=skin_mask)
        
        if augmentation:
            rotation_degrees = np.random.choice(range(-20, 21), 10, replace=False)
            for angle in rotation_degrees:
                rotated_image = rotate(skin_extracted_image, angle, reshape=False)
                rotated_image_resized = cv2.resize(rotated_image, target_size)
                skin_segmentation_images.append(rotated_image_resized.flatten())
                labels.append(y_data)
        else:
            image_resized = cv2.resize(skin_extracted_image, target_size)
            skin_segmentation_images.append(image_resized.flatten())
            labels.append(y_data)
        
    return skin_segmentation_images, labels

"""x_train_images = []
y_train_label = []
for image_path, y in zip(x_train, y_train):
    image_array, label = load_and_preprocess_image(image_path, y, augmentation=True)
    x_train_images.extend(image_array)
    y_train_label.extend(label)
x_train_images = np.array(x_train_images)"""

x_test_images = []
y_test_label = []
for image_path, y in zip(x_test, y_test):
    print(image_path)
    image_array, label = load_and_preprocess_image(image_path, y, augmentation=True)
    x_test_images.extend(image_array)
    y_test_label.extend(label)
x_test_images = np.array(x_train_images)


In [ ]:
print(np.array(x_train_prep).shape)

print(len(y_train_label))

# SVM 

In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import classification_report

# train the model on train set 
model_SVM = SVC() 
model_SVM.fit(x_train_images, y_train_label) 

# print prediction results 
#predictions = model.predict(x_test_images) 
#print(classification_report(y_test_encoded, predictions)) 

In [ ]:
predictions = model_SVM.predict(x_test_images) 
print(classification_report(y_test_encoded, predictions))

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score

precision = precision_score(y_test_encoded, predictions, average='weighted')
recall = recall_score(y_test_encoded, predictions, average='weighted')
f1 = f1_score(y_test_encoded, predictions, average='weighted')

print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)

In [ ]:
# Попробуем поменять параметры 
from sklearn.model_selection import GridSearchCV 
  
# defining parameter range 
param_grid = {'C': [0.0001, 0.001, 0.01, 0.1],  
              'gamma': [0.0001, 0.001, 0.01, 0.1], 
              'kernel': ['poly', 'linear']}  
  
grid_SVM = GridSearchCV(SVC(), param_grid, refit = True, verbose = 3) 
  
# fitting the model for grid search 
grid_SVM.fit(x_train_images, y_train_encoded) 

In [ ]:
# print best parameter after tuning 
print(grid_SVM.best_params_) 
# print how our model looks after hyper-parameter tuning 
print(grid_SVM.best_estimator_) 
grid_predictions_SVM = grid_SVM.predict(x_test_images)   
# print classification report 
print(classification_report(y_test_encoded, grid_predictions_SVM)) 

# RandomForest

In [ ]:
RANDOM_STATE = 128

In [ ]:
from sklearn.ensemble import RandomForestClassifier


model_RFC = RandomForestClassifier(random_state=RANDOM_STATE) 
model_RFC.fit(x_train_images, y_train_encoded) 

# print prediction results 
predictions_RFC = model_RFC.predict(x_test_images) 
print(classification_report(y_test_encoded, predictions_RFC)) 

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score

precision = precision_score(y_test_encoded, predictions_RFC, average='weighted')
recall = recall_score(y_test_encoded, predictions_RFC, average='weighted')
f1 = f1_score(y_test_encoded, predictions_RFC, average='weighted')

print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)

In [ ]:
param_grid = { 
    'n_estimators': [100,300,400],
    'max_features': ['sqrt'],
    'max_depth' : [1,2,5,10,30],
    'criterion' :['gini', 'entropy']
}

grid_RFC = GridSearchCV(RandomForestClassifier(), param_grid, refit = True, verbose = 3) 
grid_RFC.fit(x_train_images, y_train_encoded) 
 

In [ ]:
# print best parameter after tuning 
print(grid_RFC.best_params_) 
# print how our model looks after hyper-parameter tuning 
print(grid_RFC.best_estimator_) 
grid_predictions_RFC = grid_RFC.best_estimator_.predict(x_test_images)   
# print classification report 
print(classification_report(y_test_encoded, grid_predictions_RFC)) 

# XGBoost

In [ ]:
import xgboost as xgb
from sklearn.model_selection import GridSearchCV 
from sklearn.metrics import classification_report


xgb_model = xgb.XGBClassifier(objective="multi:softprob", random_state=42)
xgb_model.fit(x_train_images, y_train_encoded)

y_pred_XGB = xgb_model.predict(x_test_images)

print(classification_report(y_test_encoded, y_pred_XGB))


In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score

precision = precision_score(y_test_encoded, y_pred_XGB, average='weighted')
recall = recall_score(y_test_encoded, y_pred_XGB, average='weighted')
f1 = f1_score(y_test_encoded, y_pred_XGB, average='weighted')

print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)



In [ ]:
params = {
    'n_estimators': [100,300],  # Количество деревьев в ансамбле
    'max_depth': [3, 9],             # Максимальная глубина дерева
    'learning_rate': [0.1, 0.01, 0.001],# Скорость обучения
}

grid_XGB = GridSearchCV(xgb.XGBClassifier(objective="multi:softprob"), params, refit=True, verbose=2) 
grid_XGB.fit(x_train_images, y_train_encoded)

In [ ]:
# print best parameter after tuning 
print(grid_XGB.best_params_) 
# print how our model looks after hyper-parameter tuning 
print(grid_XGB.best_estimator_) 
grid_predictions_XGB = grid_XGB.best_estimator_.predict(x_test_images)   
# print classification report 
print(classification_report(y_test_encoded, grid_predictions_XGB))